In [1]:
import numpy as np
import numbers
from scipy.misc import comb

def _is_integer(val):
    return isinstance(val, (int, np.int32, np.int64))

# Python 3.5

[Used paper](http://sci-hub.io/10.1109/IJCNN.2013.6706860)

$$
K(x, y) = \sum_{r=0}^{d} (x-a)^r(y-a)^r + \sum_{r=0}^{d} \frac{1}{2d - r + 1} \begin{pmatrix} d\\r \end{pmatrix}
(|x - y|)^r(\mathrm{min}(x, y)  - a)^{2d - r + 1}
$$

$$
\mathbf{K}(\mathbf{x}, \mathbf{y}) = \prod_{j=1}^n K_j(x_j, y_j)
$$

$$
\widetilde{\mathbf{K}}(\mathbf{x}, \mathbf{y}) = \frac{\mathbf{K}(\mathbf{x}, \mathbf{y})}{\sqrt{\mathbf{K}(\mathbf{x}, \mathbf{x})
\mathbf{K}(\mathbf{y}, \mathbf{y})}}
$$

Где под INK-spline ядром понимается $\widetilde{\mathbf{K}}$

In [2]:
def K(x: np.ndarray, y: np.ndarray, degree: int, a: int) -> float:
    S = np.zeros(x.shape[0])
    for i in np.where(x < y):
        x[i], y[i] = y[i], x[i]
    for r in range(degree + 1):
        S += ((x - a)*(y - a))**r +\
        1.0/(2 * degree - r + 1) * comb(degree, r)*((x - y)**r)*(y - a)**(2*degree - r + 1)
    return np.multiply.reduce(S)

def inke(x: np.ndarray, y: np.ndarray, degree: int, a: int = -3) -> float:
    assert _is_integer(degree) and degree > 0, "Degree must be positive integer"
    assert isinstance(x, np.ndarray) and isinstance(y, np.ndarray), "X and Y must be numpy arrays"
    assert len(x.shape) == len(y.shape) == 1, "X and Y must have only one dimension"
    
    return K(x, y, degree, a) / np.sqrt(K(x, x, degree, a) * K(y, y, degree, a))

In [3]:
inke(np.array([1, 2, 3]), np.array([4, 5, 6]), 1)

0.85704509128454698

[Used paper](http://sci-hub.io/10.1109/IJCNN.2013.6706860)

$$
K(x, y) = \sum_{r=0}^{d} (x-a)^r(y-a)^r + \sum_{r=0}^{d} \frac{1}{2d - r + 1} \begin{pmatrix} d\\r \end{pmatrix}
(|x - y|)^r(\mathrm{min}(x, y)  - a)^{2d - r + 1}
$$

$$
\mathbf{K}(\mathbf{x}, \mathbf{y}) = \prod_{j=1}^n K_j(x_j, y_j)
$$

$$
\widetilde{\mathbf{K}}(\mathbf{x}, \mathbf{y}) = \frac{\mathbf{K}(\mathbf{x}, \mathbf{y})}{\sqrt{\mathbf{K}(\mathbf{x}, \mathbf{x})
\mathbf{K}(\mathbf{y}, \mathbf{y})}}
$$

Где под INK-spline ядром понимается $\widetilde{\mathbf{K}}$

In [95]:
def new_K(X: np.ndarray, Y: np.ndarray, degree: int, a: int) -> np.ndarray:
    gram = np.zeros((X.shape[0], Y.shape[0]))
    for idx, x in enumerate(X):
        S = np.zeros(Y.shape)
        for r in range(degree + 1):
            S += ((x - a)*(Y - a))**r +\
            1.0/(2 * degree - r + 1) * comb(degree, r)*((np.abs(x - Y))**r)*(np.minimum(x, Y) - a)**(2*degree - r + 1)
        gram_row = np.multiply.reduce(S, axis=1)
        gram[idx] = gram_row
    return gram

def K_norm(X: np.ndarray, degree: int, a: int, axis: str) -> np.ndarray:
    assert axis == "x" or axis =="y", "Invalid axis argument"
    gram = np.zeros((X.shape[0], X.shape[0]))
    for idx, x in enumerate(X):
        S = np.zeros(x.shape)
        for r in range(degree + 1):
            S += ((x - a)*(x - a))**r +\
            1.0/(2 * degree - r + 1) * comb(degree, r)*(0**r)*(x - a)**(2*degree - r + 1)
        S = np.multiply.reduce(S)
        gram[idx] = S
    if axis == "x":
        return gram
    if axis == "y":
        return np.transpose(gram)
        
        
def new_ink(x: np.ndarray, y: np.ndarray, degree: int, a: int = -3) -> np.ndarray:
    assert _is_integer(degree) and degree > 0, "Degree must be positive integer"
    assert isinstance(x, np.ndarray) and isinstance(y, np.ndarray), "X and Y must be numpy arrays"
    
    if len(x.shape) == 1:
        x = x.reshape(1, x.shape[0])
    if len(y.shape) == 1:
        y = y.reshape(1, y.shape[0])
    return new_K(x, y, degree, a) / np.sqrt(K_norm(x, degree, a, "x") * K_norm(y, degree, a, "y"))

In [54]:
new_ink(np.array([1, 2, 3]), np.array([4, 5, 6]), 1)

array([[ 0.85704509]])

In [55]:
import ink
import numpy.random
from functools import partial

degree = 1
former = ink.ink_svc_get(degree)
now = partial(new_ink, degree=degree)
A = numpy.array([[1, 2, 3], [4, 5, 6]])

print(former(A, A))
print(now(A, A))

[[ 1.          0.85704509]
 [ 0.85704509  1.        ]]
[[ 1.          0.85704509]
 [ 0.85704509  1.        ]]


In [111]:
A = numpy.random.randn(3, 3)

In [112]:
print(former(A, A))
print(now(A, A))

[[ 1.          0.94402594  0.95560921]
 [ 0.86207223  1.          0.95560921]
 [ 0.95610466  0.95560921  1.        ]]
[[ 1.          0.95610466  0.86207223]
 [ 0.95610466  1.          0.95560921]
 [ 0.86207223  0.95560921  1.        ]]


In [101]:
A = numpy.random.randn(4, 2)

In [102]:
print(former(A, A))
print(now(A, A))

[[ 1.          0.98390948  0.9849108   0.91920589]
 [ 0.98390948  1.          0.9849108   0.91920589]
 [ 0.99963818  0.98651964  1.          0.91920589]
 [ 0.96373011  0.99966942  0.98651964  1.        ]]
[[ 1.          0.96373011  0.96023763  0.91904612]
 [ 0.96373011  1.          0.99966942  0.98247566]
 [ 0.96023763  0.99966942  1.          0.98651964]
 [ 0.91904612  0.98247566  0.98651964  1.        ]]


In [114]:
x = A[0]
y = A[2]
inke(y, x, 1)

0.86207223437209068

In [115]:
A = numpy.random.randn(100, 2)
%timeit former(A, A)
%timeit now(A, A)

1 loop, best of 3: 1.21 s per loop
100 loops, best of 3: 13.3 ms per loop
